In [180]:
import requests
import os
from dotenv import load_dotenv, dotenv_values
import pandas as pd
import datetime
load_dotenv()

True

In [181]:
# on récupére la clé d'API stockée dans le .env
API_KEY = os.getenv("API_KEY")

Récuperation des données json de l'API geocoding d'une ville:

In [182]:
params_latlon = {
        'q' : 'Lille',
        'limit' : 1,
        'appid' : API_KEY
    }
city_geo_js = requests.get("http://api.openweathermap.org/geo/1.0/direct", params=params_latlon)
city_geo_js.json()

[{'name': 'Lille',
  'local_names': {'eo': 'Lillo',
   'zh': '里尔',
   'ur': 'لیلا نالہ',
   'ru': 'Лилль',
   'ca': 'Lilla',
   'br': 'Rysel',
   'ja': 'リール',
   'ko': '릴',
   'sr': 'Лил',
   'hu': 'Lille',
   'nl': 'Rijsel',
   'el': 'Λίλλη',
   'fr': 'Lille',
   'la': 'Insula',
   'lt': 'Lilis',
   'ar': 'ليل',
   'it': 'Lilla',
   'uk': 'Лілль'},
  'lat': 50.6365654,
  'lon': 3.0635282,
  'country': 'FR',
  'state': 'Hauts-de-France'}]

Obtenir la latitude et la longitude d'une ville

In [183]:
f"Latitude : {city_geo_js.json()[0]['lat']}, Longitude : {city_geo_js.json()[0]['lon']}"

'Latitude : 50.6365654, Longitude : 3.0635282'

Obtenir les données json de l'API météo d'une ville :

In [184]:
lat = city_geo_js.json()[0]['lat']
lon = city_geo_js.json()[0]['lon']

params = {
        'lat' : lat,
        'lon': lon,
        'units' : 'metric',
        'appid' : API_KEY
    }

response = requests.get("https://api.openweathermap.org/data/2.5/weather", params=params)
response.json()

{'coord': {'lon': 3.0635, 'lat': 50.6366},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 11.65,
  'feels_like': 9.94,
  'temp_min': 9.64,
  'temp_max': 12.7,
  'pressure': 1026,
  'humidity': 41},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 40},
 'clouds': {'all': 0},
 'dt': 1680532581,
 'sys': {'type': 2,
  'id': 2011132,
  'country': 'FR',
  'sunrise': 1680499215,
  'sunset': 1680546101},
 'timezone': 7200,
 'id': 6454414,
 'name': 'Lille',
 'cod': 200}

Définition du dataframe

In [185]:
labels = [
        ['Temperature', 'Temperature', 'Temperature', 'Temperature', '', 'Pression', '', '', '', 'Vent', 'Vent', '', 'Soleil', 'Soleil'], 
        ['Actuelle', 'Ressentie', 'Minimale', 'Maximale', '', 'Atmospherique', '', 'Humidité', '', 'Vitesse', 'Direction', '', 'Lever', 'Coucher']]
tuples = list(zip(*labels))
index = pd.MultiIndex.from_tuples(tuples, names=['',''])

df = pd.DataFrame([[]], index=index)
df

Empty DataFrame
Columns: []
Index: [(Temperature, Actuelle), (Temperature, Ressentie), (Temperature, Minimale), (Temperature, Maximale), (, ), (Pression, Atmospherique), (, ), (, Humidité), (, ), (Vent, Vitesse), (Vent, Direction), (, ), (Soleil, Lever), (Soleil, Coucher)]

Preparation

In [186]:
# on repertorie les 20 plus grandes villes de France sur lesquelles nous souhaitons travailler
# Note : les noms ont le format nécessaire pour les instancier depuis l'API
cities = [
    'Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Strasbourg',  'Bordeaux', 'Lille', 'Montpellier', 'Nantes',
    'Rennes', 'Toulon', 'Grenoble', 'Reims', 'Saint-Etienne', 'Angers', 'Dijon', 'Le_Havre', 'Nimes', 'Villeurbanne']

# ce dictionnaire servira pour convertir la direction en dégré du vent en point cardinal
# Avec la formule : ( direction_du_vent / 22.5) + 1 on obtiendra un chiffre entre 1 et 17
secteurs = {1:"N",2:"NNE",3:"NE",4:"ENE",5:"E",6:"ESE",7:"SE",8:"SSE",9:"S",10:"SSW",11:"SW",12:"WSW",13:"W",14:"WNW",15:"NW",16:"NNW",17:"N"}

In [187]:
#on loop sur toutes les villes
for city in cities:
    #on recupere les latitudes et longitudes de chaque ville
    params_latlon = {
        'q' : city,
        'limit' : 1,
        'appid' : API_KEY
    }
    city_geo_js = requests.get("http://api.openweathermap.org/geo/1.0/direct", params=params_latlon)
    lat = city_geo_js.json()[0]['lat']
    lon = city_geo_js.json()[0]['lon']

    #on genere les données pour chaque ville
    params = {
        'lat' : lat,
        'lon': lon,
        'units' : 'metric',
        'appid' : API_KEY
    }
    response = requests.get("https://api.openweathermap.org/data/2.5/weather", params=params)
    city_js = response.json()

    #on crée une colonne comportant son nom puis toutes les valeurs qui nous interessent
    df[city] = [
        str(city_js['main']['temp'])+'°C',
        str(city_js['main']['feels_like'])+'°C',
        str(city_js['main']['temp_min'])+'°C',
        str(city_js['main']['temp_max'])+'°C',
        '',
        str(city_js['main']['pressure'])+'hPa',
        '',
        str(city_js['main']['humidity'])+'%',
        '',
        str(round(city_js['wind']['speed']*3.6))+'km/h',
        secteurs[round((city_js['wind']['deg']/22.5)+1)],
        '',
        datetime.datetime.fromtimestamp(city_js['sys']['sunrise']).strftime("%H:%M:%S"),
        datetime.datetime.fromtimestamp(city_js['sys']['sunset']).strftime("%H:%M:%S")
    ]
    
    #On renomme les villes au bon format, avec espaces et/ou accents
    df = df.rename(columns={'Saint-Etienne':'Saint-Étienne', 'Le_Havre':'Le Havre', 'Nimes':'Nîmes'})
pd.set_option('display.max_columns', None)
df = df.transpose()
df

Temperature                                    Pression    \
                 Actuelle Ressentie Minimale Maximale   Atmospherique     
Paris             11.99°C   10.55°C  11.16°C  13.24°C         1024hPa     
Marseille         19.81°C   18.79°C  16.78°C  20.95°C         1010hPa     
Lyon              10.33°C    9.38°C    9.4°C  10.55°C         1018hPa     
Toulouse          13.07°C   12.05°C  12.97°C  14.32°C         1019hPa     
Nice               16.1°C   15.65°C  15.92°C  18.91°C         1009hPa     
Strasbourg         7.88°C    4.15°C   7.12°C   8.47°C         1025hPa     
Bordeaux          14.21°C    13.1°C  14.21°C  15.25°C         1020hPa     
Lille             11.65°C    9.94°C   9.64°C   12.7°C         1026hPa     
Montpellier       18.16°C   17.02°C  17.27°C  18.59°C         1012hPa     
Nantes             9.48°C    6.89°C   9.06°C  11.04°C         1023hPa     
Rennes            10.96°C    9.76°C  10.94°C  12.46°C         1024hPa     
Toulon            18.63°C   18.01°C  16.71°C  22.64°C         1008hPa     
Grenoble          10.11°C     9.4°C   8.29°C  12.39°C         1016hPa     
Reims                11°C    9.46°C     11°C  11.26°C         1025hPa     
Saint-Étienne      7.17°C    3.85°C   7.17°C    8.2°C         1019hPa     
Angers            13.93°C   12.66°C  11.16°C  13.93°C         1023hPa     
Dijon              8.84°C    5.89°C   8.64°C  10.13°C         1021hPa     
Le Havre          11.51°C   10.23°C  10.86°C  11.51°C         1026hPa     
Nîmes             17.05°C   15.83°C  15.85°C  18.55°C         1013hPa     
Villeurbanne      10.35°C    9.37°C   9.64°C  10.59°C         1018hPa     

                            Vent                Soleil            
              Humidité   Vitesse Direction       Lever   Coucher  
Paris              50%    26km/h       ENE    07:24:58  20:22:55  
Marseille          36%    32km/h        NW    07:17:31  20:05:58  
Lyon               75%    30km/h       NNW    07:17:42  20:10:06  
Toulouse           62%    22km/h        NW    07:32:58  20:21:55  
Nice               72%    13km/h         S    07:09:37  19:58:41  
Strasbourg         49%    26km/h       ENE    07:03:32  20:00:55  
Bordeaux           54%    15km/h        NE    07:40:04  20:31:00  
Lille              41%    22km/h        NE    07:20:15  20:21:41  
Montpellier        38%    24km/h       NNE    07:23:14  20:12:11  
Nantes             73%    19km/h       ENE    07:41:57  20:36:55  
Rennes             63%     9km/h         E    07:41:38  20:38:14  
Toulon             56%    28km/h        SW    07:15:24  20:03:36  
Grenoble           85%    19km/h         N    07:14:33  20:06:00  
Reims              50%    20km/h       ENE    07:17:45  20:16:26  
Saint-Étienne      81%    20km/h         N    07:19:44  20:11:37  
Angers             49%    20km/h        NE    07:37:43  20:33:08  
Dijon              66%    20km/h        NE    07:15:30  20:10:36  
Le Havre           58%    19km/h       ENE    07:33:11  20:32:23  
Nîmes              39%    35km/h         N    07:21:08  20:10:26  
Villeurbanne       74%    33km/h         N    07:17:28  20:09:53

Enregistrer en csv

In [188]:
df.to_csv('OpenWeather_20_Villes_Francaises.csv')